In [1]:
import numpy as np
import pandas as pd

In [21]:
ipynb_jupyter_repos = pd.read_parquet('../data/ipynb_component_counter_jupyter_repos.parquet')
ipynb_jupyter_repos

,filename,module,component_type,component_name,count
0,/media/tobiasz/crucial/jupyter_repos/-PyTorch-...,math,function,sqrt,2
1,/media/tobiasz/crucial/jupyter_repos/-PyTorch-...,copy,function,deepcopy,1
2,/media/tobiasz/crucial/jupyter_repos/-PyTorch-...,os,attribute,path,2
3,/media/tobiasz/crucial/jupyter_repos/02_Python...,random,function,shuffle,1
4,/media/tobiasz/crucial/jupyter_repos/02_Python...,decimal,class,Decimal,1
...,...,...,...,...,...
119732,/media/tobiasz/crucial/jupyter_repos/ztdl-mast...,base64,function,b64encode,1
119733,/media/tobiasz/crucial/jupyter_repos/ztdl-mast...,io,method,getvalue,1
119734,/media/tobiasz/crucial/jupyter_repos/ztdl-mast...,gzip,function,open,2
119735,/media/tobiasz/crucial/jupyter_repos/ztdl-mast...,gzip,function,open,2


In [27]:
py_jupyter_repos = pd.read_parquet('../data/py_component_counter_jupyter_repos.parquet')
py_jupyter_repos

,filename,module,component_type,component_name,count
0,/media/tobiasz/crucial/jupyter_repos/-PyTorch-...,xml.etree.ElementTree,method,iter,1
1,/media/tobiasz/crucial/jupyter_repos/-PyTorch-...,os,attribute,path,5
2,/media/tobiasz/crucial/jupyter_repos/-PyTorch-...,xml.etree.ElementTree,attribute,tag,6
3,/media/tobiasz/crucial/jupyter_repos/-PyTorch-...,xml.etree.ElementTree,method,append,2
4,/media/tobiasz/crucial/jupyter_repos/-PyTorch-...,xml.etree.ElementTree,attribute,text,4
...,...,...,...,...,...
285187,/media/tobiasz/crucial/jupyter_repos/zoobot/be...,os,function,mkdir,1
285188,/media/tobiasz/crucial/jupyter_repos/zoobot/be...,os,function,listdir,2
285189,/media/tobiasz/crucial/jupyter_repos/zoobot/be...,os,attribute,path,5
285190,/media/tobiasz/crucial/jupyter_repos/ztdl-mast...,tempfile,function,NamedTemporaryFile,1


In [23]:
ipynb_jupyter_repos['repo'] = ipynb_jupyter_repos['filename'].str.split('/').str[5]
ipynb_jupyter_repos['filename'] = ipynb_jupyter_repos.apply(lambda row: row['filename'].split(row['repo'])[-1], axis=1).str[1:]
ipynb_jupyter_repos = ipynb_jupyter_repos[['repo', 'filename', 'module', 'component_type', 'component_name', 'count']]

In [26]:
ipynb_jupyter_repos

,repo,filename,module,component_type,component_name,count
0,-PyTorch-,07_注意力自注意力多头注意力/attention.ipynb,math,function,sqrt,2
1,-PyTorch-,07_注意力自注意力多头注意力/attention.ipynb,copy,function,deepcopy,1
2,-PyTorch-,01_手把手教你跑通第一个神经网络/01_手把手教你跑通第一个神经网络.ipynb,os,attribute,path,2
3,02_Python_Datatypes,001_Python_Numbers.ipynb,random,function,shuffle,1
4,02_Python_Datatypes,001_Python_Numbers.ipynb,decimal,class,Decimal,1
...,...,...,...,...,...,...
119732,ztdl-masterclasses,notebooks/Pre-trained_Models.ipynb,base64,function,b64encode,1
119733,ztdl-masterclasses,notebooks/Pre-trained_Models.ipynb,io,method,getvalue,1
119734,ztdl-masterclasses,notebooks/Natural_Language_Processing.ipynb,gzip,function,open,2
119735,ztdl-masterclasses,notebooks/Word_Embeddings.ipynb,gzip,function,open,2


In [28]:
py_jupyter_repos['repo'] = py_jupyter_repos['filename'].str.split('/').str[5]
py_jupyter_repos['filename'] = py_jupyter_repos.apply(lambda row: row['filename'].split(row['repo'])[-1], axis=1).str[1:]
py_jupyter_repos = py_jupyter_repos[['repo', 'filename', 'module', 'component_type', 'component_name', 'count']]

In [41]:
len(ipynb_jupyter_repos['filename'].unique())

30716

In [34]:
unique_repos = np.unique(np.concatenate((ipynb_jupyter_repos['repo'].unique(), py_jupyter_repos['repo'].unique())))